In [ ]:
###################################################### PASO 00 ######################################################

                                         ###  LIBRERIAS

import os
import datetime
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter("ignore")

###################################################### PASO 01 ######################################################

                                         ###  RUTAS DE ENTRADA Y SALIDA (PATH)


Entradas = '---'


Salidas = '----'

In [2]:
###################################################### PASO 02 ######################################################

                                         ### ITV_OBJ_SUP

### Se carga la tabla "ITV_OBJ_SUP"

ITV_OBJ_SUP = pd.read_csv(Salidas + '/ITV_OBJ_SUP.csv', encoding = 'Latin-1')
print(ITV_OBJ_SUP.shape)

(3810, 39)


In [3]:
###################################################### PASO 03 ######################################################

                                         ### BASE_UA

### Se crea la tabla "BASE_UA" a partir del cálculo en "ADM_GRAL" con agrupación de "ITV_OBJ_SUP" en:
                             ### --> 'UNIDAD_ADM','REGION', 'SEDE','UNIDAD', 'LOCAL'
                             ### --> 'AMBITO', 'AG_SUPERVISADAS_UA','UA_PTO','UA_EMP','UA_MIARI'
                     ### --> Para "ADM_GRAL" se cuentan los elementos distintos
                     ### --> Se ordenan los valores con base a "UA_MIARI" de manera descendente
            
BASE_UA = ITV_OBJ_SUP.groupby(['UNIDAD_ADM', 
                     'REGION', 'SEDE', 
                     'UNIDAD', 'LOCAL', 
                     'AMBITO', 'AG_SUPERVISADAS_UA', 
                     'UA_PTO','UA_EMP', 
                     'UA_MIARI' ],dropna=False).agg({'ADM_GRAL':'nunique'}).reset_index().sort_values(by=['UA_MIARI'],
                                                                                                      ascending=[False])

### Se renombra la columna calculada:

BASE_UA = BASE_UA.rename({'ADM_GRAL':'COUNT_ADM_GRAL'}, axis=1)

print(BASE_UA.shape)

(157, 11)


In [4]:
###################################################### PASO 04 ######################################################

                                         ### BASE_UA_AG

### Se crea la tabla "BASE_UA" a partir del cálculo en "SUBPROCESO" con agrupación de "ITV_OBJ_SUP" en:
                             ### --> 'UNIDAD_ADM', 'REGION','SEDE','UNIDAD', 'LOCAL','AG_SUPERVISADAS_UA',
                             ### --> 'UA_PTO','UA_EMP','UA_MIARI', 'AMBITO','ADM_GRAL',
                             ### --> 'SUPERVISION', 'UA_AG_PTO','UA_AG_EMP', 'UA_AG_MIARI'
                     ### --> Para "SUBPROCESO" se cuentan los elementos distintos
                     ### --> Se ordenan los valores con base a "UA_MIARI" de manera descendente

BASE_UA_AG = ITV_OBJ_SUP.groupby(['UNIDAD_ADM', 'REGION',
                                  'SEDE', 'UNIDAD', 'LOCAL', 
                                  'AG_SUPERVISADAS_UA', 'UA_PTO','UA_EMP', 
                                  'UA_MIARI', 'AMBITO','ADM_GRAL',
                                  'SUPERVISION', 'UA_AG_PTO', 
                                  'UA_AG_EMP', 'UA_AG_MIARI'],dropna=False).agg({'SUBPROCESO':'nunique'}).reset_index().sort_values(by=['UA_AG_MIARI'],
                                                                                                                                    ascending=[False])
### Se renombra la columna calculada:

BASE_UA_AG = BASE_UA_AG.rename({'SUBPROCESO':'COUNT_SUBPROCESO'}, axis=1)

print(BASE_UA_AG.shape)

(320, 16)


In [5]:
###################################################### PASO 05 ######################################################

                                         ### RANK_AG_UA
    
### Se cálcula "RANK_AR" para "AMBITO == DESCONCENTRADA", "AMBITO == CENTRAL", "AMBITO == CENTRAL" y "OTROS" a partir de:

### ---> DESCONCENTRADA
### Si "[AMBITO == DESCONCENTRADA] & [ADM_GRAL == (AGACE,AGJ, AGAFF, AGRS,AGR,AGCTI, AGSC)]", entonces "suma acumulativa"
    ### --> Si lo anterior no se cumple, entonces para despues del primer caso donde es "0" el valor es el anterior "RANK_AR" 

A = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGACE'))
BASE_UA_AG['RANK_AR'] = np.where(A, A.cumsum(), 0)

B = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGJ'))
BASE_UA_AG['RANK_AR'] = np.where(B, B.cumsum(), BASE_UA_AG.RANK_AR)

C = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGAFF'))
BASE_UA_AG['RANK_AR'] = np.where(C, C.cumsum(), BASE_UA_AG.RANK_AR)

D = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGRS'))
BASE_UA_AG['RANK_AR'] = np.where(D, D.cumsum(), BASE_UA_AG.RANK_AR)

E = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGR'))
BASE_UA_AG['RANK_AR'] = np.where(E, E.cumsum(), BASE_UA_AG.RANK_AR)

F = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGCTI'))
BASE_UA_AG['RANK_AR'] = np.where(F, F.cumsum(), BASE_UA_AG.RANK_AR)

G = ((BASE_UA_AG.AMBITO == 'DESCONCENTRADA') & (BASE_UA_AG.ADM_GRAL  == 'AGSC'))
BASE_UA_AG['RANK_AR'] = np.where(G, G.cumsum(), BASE_UA_AG.RANK_AR)


### ---> CENTRAL
### Si "[AMBITO == CENTRAL] & [ADM_GRAL == (AGACE,AGE,AGJ,AGGC,AGAFF AGRS,AGR,AGCTI,AGSC,AGH,AGP)]", entonces "suma acumulativa"
    ### --> Si lo anterior no se cumple, entonces el valor es el anterior "RANK_AR" 
    
H = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGACE'))
BASE_UA_AG['RANK_AR'] = np.where(H, H.cumsum(), BASE_UA_AG.RANK_AR)

I = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGE'))
BASE_UA_AG['RANK_AR'] = np.where(I, I.cumsum(), BASE_UA_AG.RANK_AR)

J = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGJ'))
BASE_UA_AG['RANK_AR'] = np.where(J, J.cumsum(), BASE_UA_AG.RANK_AR)

K = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGGC'))
BASE_UA_AG['RANK_AR'] = np.where(K, K.cumsum(), BASE_UA_AG.RANK_AR)

L = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGAFF'))
BASE_UA_AG['RANK_AR'] = np.where(L, L.cumsum(), BASE_UA_AG.RANK_AR)

M = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGRS'))
BASE_UA_AG['RANK_AR'] = np.where(M, M.cumsum(), BASE_UA_AG.RANK_AR)

N = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGR'))
BASE_UA_AG['RANK_AR'] = np.where(N, N.cumsum(), BASE_UA_AG.RANK_AR)

O = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGA'))
BASE_UA_AG['RANK_AR'] = np.where(O, O.cumsum(), BASE_UA_AG.RANK_AR)

P = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGCTI'))
BASE_UA_AG['RANK_AR'] = np.where(P, P.cumsum(), BASE_UA_AG.RANK_AR)

Q = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGSC'))
BASE_UA_AG['RANK_AR'] = np.where(Q, Q.cumsum(), BASE_UA_AG.RANK_AR)

R = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGH'))
BASE_UA_AG['RANK_AR'] = np.where(R, R.cumsum(), BASE_UA_AG.RANK_AR)

S = ((BASE_UA_AG.AMBITO == 'CENTRAL') & (BASE_UA_AG.ADM_GRAL  == 'AGP'))
BASE_UA_AG['RANK_AR'] = np.where(S, S.cumsum(), BASE_UA_AG.RANK_AR)


### ---> ADUANA
### Si "[AMBITO == ADUANA] & [ADM_GRAL == (AGA,AGCTI,AGRS)]", entonces "suma acumulativa"
    ### --> Si lo anterior no se cumple, entonces el valor es el anterior "RANK_AR" 
    
T = ((BASE_UA_AG.AMBITO == 'ADUANA') & (BASE_UA_AG.ADM_GRAL  == 'AGA'))
BASE_UA_AG['RANK_AR'] = np.where(T, T.cumsum(), BASE_UA_AG.RANK_AR)

U = ((BASE_UA_AG.AMBITO == 'ADUANA') & (BASE_UA_AG.ADM_GRAL  == 'AGCTI'))
BASE_UA_AG['RANK_AR'] = np.where(U, U.cumsum(), BASE_UA_AG.RANK_AR)

V = ((BASE_UA_AG.AMBITO == 'ADUANA') & (BASE_UA_AG.ADM_GRAL  == 'AGRS'))
BASE_UA_AG['RANK_AR'] = np.where(V, V.cumsum(), BASE_UA_AG.RANK_AR)


### ---> GENERAL & REGIONAL
### Para todos los casos donde "RANK_AR" es igual a "0" y donde "[AMBITO == (GENERAL,REGIONAL)]"
    ### --> Si lo anterior no se cumple, entonces el valor es el anterior "RANK_AR" 
    
W = (BASE_UA_AG.RANK_AR == 0)
BASE_UA_AG['RANK_AR'] =np.where(W, W.cumsum(), BASE_UA_AG.RANK_AR)

print(BASE_UA_AG.shape)

(320, 17)


In [6]:
###################################################### PASO 06 ######################################################

                                         ### RANK_UA_DES_AG

### Se crea la tabla "RANK_UA_DES_AG" a partir del cálculo de un nuevo nivel con base a: "RANK_AR" y "ADM_GRAL"
                             ### --> "RANK_AR" y "ADM_GRAL"
            ### --> Este proceso se considera como una matriz transpuesta agrupada en una columna
### La tabla "RK_UA_DES_AG" contiene los campos:
                             ### --> 'UNIDAD_ADM','REGION','SEDE','UNIDAD','LOCAL','ADM_GRAL','RANK_AR'

RK_UA_DES_AG = BASE_UA_AG[['UNIDAD_ADM','REGION','SEDE','UNIDAD','LOCAL','ADM_GRAL','RANK_AR']]

### Se renombra la columna a transponer por facilidad de procesos siguientes:

RK_UA_DES_AG = RK_UA_DES_AG.rename({"RANK_AR": "X_"},axis = 1)

### Se traspone (unstack) el dataframe para la columna "RANK_AR(X_)" como suma, agrupando por:
                             ### --> 'UNIDAD_ADM','REGION','SEDE','UNIDAD','LOCAL','ADM_GRAL'
    
RANK_UA_DES_AG = RK_UA_DES_AG.groupby(['UNIDAD_ADM','REGION',
                                       'SEDE','UNIDAD','LOCAL',
                                       'ADM_GRAL'], sort=False).sum().unstack().reset_index()

### El dataframe transpuesto es de orden multinivel por la información en "RANK_AR(X_)", se restructura de la forma:
                             ### --> Se extrae el nombre de las columnas y se guardan como lista "CL_MI.tolist()"
                                       ### --> ('X_','AGAFF') [Columna de nombre multinivel (a,b)]
                             ### --> La lista de nombres se aplanan y transforman en índices
                                       ### --> ('X_','AGAFF') ==> 'X_AGAFF'     
                             ### --> Los índices se convierten ahora en el nombre de las columnas
                             ### --> Se eliminan los subfijos en los nombres de las columnas (X_)

CL_MI = RANK_UA_DES_AG.columns
IND = pd.Index([e[0] + e[1] for e in CL_MI.tolist()])
RANK_UA_DES_AG.columns = IND
RANK_UA_DES_AG.columns = RANK_UA_DES_AG.columns.str.strip('X_')


### La tabla "RANK_UA_DES_AG" se ordena "UNIDAD_ADM" de forma ascendente 
### Se ordenan las columnas con base al resultado en "SAS"

RANK_UA_DES_AG = RANK_UA_DES_AG.sort_values(by=['UNIDAD_ADM'],ascending=[True]).reset_index(drop=True)
RANK_UA_DES_AG = RANK_UA_DES_AG[['UNIDAD_ADM','AGACE','AGR','AGRS',
                                 'AGCTI','AGE','AGAFF','AGGC','AGH',
                                 'AGSC','AGJ','AGP']]

### Para "RANK_UA_DES_AG" se crea el campo "AG_UA" con base al conteo de los elementos en:
                             ### --> 'AGACE','AGR','AGRS','AGCTI','AGE','AGAFF','AGGC','AGH','AGSC','AGJ','AGP'

RANK_UA_DES_AG['AG_UA'] = RANK_UA_DES_AG.iloc[:,1:12].count(axis=1)
     
print(RANK_UA_DES_AG.shape)

(157, 13)


In [7]:
###################################################### PASO 07 ######################################################

                                         ### IRS_UA_RANK

### Se crea la tabla "IRS_UA_RANK" con base la unión en:
                             ### --> "BASE_UA", "RANK_UA_DES_AG" y "IRN_RECAUDACION_UA" con campo llave "UNIDAD_ADM" 
                                      ### --> Nota: falta la tabla "IRN_RECAUDACION_UA"

IRS_UA_RANK = BASE_UA.merge(RANK_UA_DES_AG,
                            how = 'left',
                            left_on = ['UNIDAD_ADM'],
                            right_on = ['UNIDAD_ADM'])

### Para "IRS_UA_RANK" se crea el campo "UNIDAD_LOCAL" a partir de la union de textos en las columnas:
                             ### --> "UNIDAD" + "(LOCAL)"

IRS_UA_RANK["UNIDAD_LOCAL"] = IRS_UA_RANK['UNIDAD'].astype(str) +"("+ IRS_UA_RANK["LOCAL"].astype(str) +")"

### Se ordenan los campos con base a los resultados en "SAS"

IRS_UA_RANK = IRS_UA_RANK[['UNIDAD_ADM','REGION','SEDE','UNIDAD','LOCAL','UNIDAD_LOCAL','AMBITO',
                           'AG_UA','AG_SUPERVISADAS_UA','UA_PTO','UA_EMP','UA_MIARI',
                           'AGSC','AGAFF','AGGC','AGACE','AGH','AGR','AGJ','AGRS','AGCTI','AGE','AGP']]


### Se exporta "IRS_UA_RANK"
IRS_UA_RANK.to_csv(Salidas + '/IRS_UA_RANK.csv',
                        encoding = 'Latin-1', index = False)

print(IRS_UA_RANK.shape)

(157, 23)
